## Image Similarity Using PyTorch - Cosine Similarity

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

## ResNet50 Model

In [8]:
#model = models.resnet18(pretrained=True)
model = models.resnet50(pretrained=True)

In [9]:
layer = model._modules.get('avgpool')

In [4]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
  

## Rescaling and Transformation

In [6]:
scaler = transforms.Scale((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [10]:
def get_vector(image_name):
    
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    
    
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    
    
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 2048
    my_embedding = torch.zeros(2048)
    
    
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    
    
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    
    
    # 6. Run the model on our transformed image
    model(t_img)
    
    
    # 7. Detach our copy function from the layer
    h.remove()
    
    
    # 8. Return the feature vector
    
    return my_embedding

In [11]:
pic_one = get_vector('sim_images/1.jpg')
pic_two = get_vector('sim_images/2.jpg')
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos_sim = cos(pic_one.unsqueeze(0),
              pic_two.unsqueeze(0))
print('\nCosine similarity: {0}\n'.format(cos_sim))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: src is not broadcastable to dst, but they have the same number of elements.  Falling back to deprecated pointwise behavior.



Cosine similarity: 
 0.9960
[torch.FloatTensor of size 1]


